In [133]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import numpy as np
import datetime as dt
import math
import random
import time

In [134]:
data=pd.read_csv('final_large.csv')  

In [135]:
data = data.drop(columns=['Unnamed: 0'])

In [136]:
data['Volume']=data['Volume'].apply(np.floor)

In [137]:
data['Date'] = pd.to_datetime(data['Date'],format='%Y-%m-%d')

In [138]:
data = data.drop(data.loc[(data.profit<=0)].index)

In [139]:
data["vol_prof"]=data["Volume"]*data["profit"]

In [140]:
data

,Date,Open,High,Low,Close,Volume,stock,OP_HI,LO_CL,profit,vol_prof
1,1970-01-02,0.7383,0.7538,0.7383,0.7538,8378.0,CVX,0.000579,0.000579,0.000579,4.850862
5,1970-01-02,25.1260,25.8050,25.0320,25.7080,2942.0,C,0.169690,0.168600,0.169690,499.227980
6,1970-01-02,167.2500,173.9100,167.2500,173.9100,930.0,NAV,3.248400,3.248400,3.248400,3021.012000
7,1970-01-05,174.7400,184.7200,174.7400,182.2300,1770.0,NAV,6.385400,3.920300,6.385400,11302.158000
10,1970-01-05,1.4972,1.5297,1.4972,1.5297,233857.0,XOM,0.002231,0.002231,0.002231,521.734967
...,...,...,...,...,...,...,...,...,...,...,...
1340661,2017-11-10,3.1000,3.1800,2.9900,3.0100,2537.0,CLBS,0.017200,-0.040000,0.017200,43.636400
1340662,2017-11-10,1.4100,1.5100,1.4000,1.4700,32060.0,SKLN,0.070800,0.041300,0.070800,2269.848000
1340663,2017-11-10,155.0000,158.8000,155.0000,158.4500,20510.0,VMI,0.662000,0.315500,0.662000,13577.620000
1340665,2017-11-10,213.6000,221.6700,213.3000,221.0500,115944.0,ANET,3.717300,3.406500,3.717300,430998.631200


In [151]:
Start =pd.to_datetime("1970-01-02",format='%Y-%m-%d')
End = pd.to_datetime("2017-11-10",format='%Y-%m-%d')

In [156]:
sliding_window=1
window=dt.timedelta(days=sliding_window)
balance = 1.0

In [157]:
def find_best_in_company(data,balance):
    for i, row in data.iterrows():
        max_profit = float("-inf")
        low = data.loc[i]["Low"]*1.01
        max_volume=0
        final_action=0
        date=None
        business=False
        action=0
        if(data.loc[i]["Low"]*1.01<balance):
            if (data.loc[i]["OP_HI"]>=data.loc[i]["LO_CL"]): 
                action=1
                low = data.loc[i]["Open"]*1.01
            volume=min(math.ceil(balance/low)-1,data.loc[i]['Volume'])
            profit=data.loc[i]["profit"]
            if(volume>0):
                business=True
                max_profit = volume*profit
                max_volume = volume
                date=data.loc[i]['Date']
                final_action=action
                balance=balance-low*volume
    return (business,max_profit,max_volume,date,final_action,balance)

In [158]:
def find_best_in_window(data,balance,f,moves):
    new_data=data.sort_values(by="vol_prof",ascending=False)
    companies = []
    new_data=new_data[:130]
    pseudobalance=balance
    for company in new_data.stock.unique():
        hap,max_diff,volume,date,action,pseudobalance = find_best_in_company(new_data[new_data.stock == company],pseudobalance)
        if(hap):
            balance=balance+max_diff
            moves=moves+2
            if(action==1):
                f.write('{} {} {} {}\n'.format(date.date(), "buy-open", company, int(volume)))
                f.write('{} {} {} {}\n'.format(date.date(), "sell-high", company, int(volume)))
            else:
                f.write('{} {} {} {}\n'.format(date.date(), "buy-low", company, int(volume)))
                f.write('{} {} {} {}\n'.format(date.date(), "sell-close", company, int(volume)))
    return balance,moves

In [159]:
starting_window = Start
moves=0
with open('C:/Users/User/Desktop/time-travel/large.txt', 'w+') as f:
    while starting_window <= End:
        if(moves%100==0):
            print("Moves:",moves)
            print("Balance",balance)
            print("Date",starting_window.date(),"\n")
        mask = (data['Date'] ==starting_window)
        balance,moves = find_best_in_window(data.loc[mask],balance,f,moves)
        starting_window = starting_window + window
print("Total Moves=", moves)

Moves: 0
Balance 1.0
Date 1970-01-02 

Moves: 100
Balance 1.6281494999999993
Date 1970-06-19 

Moves: 200
Balance 2.5280810999999987
Date 1970-10-06 

Moves: 200
Balance 2.5280810999999987
Date 1970-10-07 

Moves: 200
Balance 2.5280810999999987
Date 1970-10-08 

Moves: 300
Balance 3.4689150999999976
Date 1971-03-02 

Moves: 300
Balance 3.4689150999999976
Date 1971-03-03 

Moves: 300
Balance 3.4689150999999976
Date 1971-03-04 

Moves: 400
Balance 4.642953099999994
Date 1971-08-26 

Moves: 400
Balance 4.642953099999994
Date 1971-08-27 

Moves: 400
Balance 4.642953099999994
Date 1971-08-28 

Moves: 400
Balance 4.642953099999994
Date 1971-08-29 

Moves: 400
Balance 4.642953099999994
Date 1971-08-30 

Moves: 400
Balance 4.642953099999994
Date 1971-08-31 

Moves: 400
Balance 4.642953099999994
Date 1971-09-01 

Moves: 400
Balance 4.642953099999994
Date 1971-09-02 

Moves: 400
Balance 4.642953099999994
Date 1971-09-03 

Moves: 500
Balance 6.005202099999989
Date 1972-03-15 

Moves: 500
Balance 

In [160]:
balance

121776455955.61334